In [1]:
from quantopian.pipeline import Pipeline, CustomFactor
from quantopian.research import run_pipeline
from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import AverageDollarVolume

In [44]:
pipe = Pipeline()
my_pipe = run_pipeline(pipe,'2016-01-01','2017-01-01').head()
my_pipe

Empty DataFrame
Columns: []
Index: [(2016-01-04 00:00:00+00:00, Equity(2 [ARNC])), (2016-01-04 00:00:00+00:00, Equity(21 [AAME])), (2016-01-04 00:00:00+00:00, Equity(24 [AAPL])), (2016-01-04 00:00:00+00:00, Equity(25 [ARNC_PR])), (2016-01-04 00:00:00+00:00, Equity(31 [ABAX]))]

In [40]:
pipe = Pipeline(columns={'price':USEquityPricing.close.latest})
my_pipe = run_pipeline(pipe,'2016-01-01','2017-01-01')
my_pipe

price
2016-01-04 00:00:00+00:00 Equity(2 [ARNC])           9.860
                          Equity(21 [AAME])          4.990
                          Equity(24 [AAPL])        105.230
                          Equity(25 [ARNC_PR])         NaN
                          Equity(31 [ABAX])         55.690
                          Equity(39 [DDC])          10.230
                          Equity(41 [ARCB])         21.380
                          Equity(52 [ABM])          28.470
                          Equity(53 [ABMD])         90.240
                          Equity(62 [ABT])          44.910
                          Equity(64 [ABX])           7.380
                          Equity(66 [AB])           23.860
                          Equity(67 [ADSK])         60.920
                          Equity(69 [ACAT])         16.379
                          Equity(70 [VBF])          17.629
                          Equity(76 [TAP])          93.920
                          Equity(84 [ACET])         26.980
                          Equity(86 [ACG])           7.670
                          Equity(88 [ACI])           0.998
                          Equity(100 [IEP])         61.300
                          Equity(106 [ACU])         17.400
                          Equity(110 [ACXM])        20.920
                          Equity(112 [ACY])         12.700
                          Equity(114 [ADBE])        93.910
                          Equity(117 [AEY])          1.810
                          Equity(122 [ADI])         55.320
                          Equity(128 [ADM])         36.670
                          Equity(134 [SXCL])        14.710
                          Equity(149 [ADX])         12.820
                          Equity(153 [AE])          39.350
...                                                    ...
2017-01-03 00:00:00+00:00 Equity(50534 [HGV])       25.983
                          Equity(50535 [PK])        30.000
                          Equity(50537 [WRD])       14.610
                          Equity(50539 [TIG])       14.600
                          Equity(50540 [TRVG])      11.740
                          Equity(50541 [KEY_PRI])   26.120
                          Equity(50542 [JHMD])      24.920
                          Equity(50543 [COWZ])      24.640
                          Equity(50544 [KEG])       31.950
                          Equity(50546 [NURE])         NaN
                          Equity(50547 [YTRA])       9.350
                          Equity(50548 [RXII_W])     0.200
                          Equity(50549 [BMLP])      52.250
                          Equity(50550 [WBIY])      24.650
                          Equity(50552 [FIV])       10.120
                          Equity(50554 [HEBT])       4.950
                          Equity(50555 [UUUU_WS])      NaN
                          Equity(50556 [SAB])       25.850
                          Equity(50558 [INFR])      25.250
                          Equity(50559 [VVPR])       5.500
                          Equity(50560 [XK])        15.030
                          Equity(50561 [XE])        15.060
                          Equity(50562 [XH])           NaN
                          Equity(50563 [XM])           NaN
                          Equity(50564 [XD])           NaN
                          Equity(50565 [XI])           NaN
                          Equity(50566 [XU])           NaN
                          Equity(50567 [XF])           NaN
                          Equity(50568 [XS])           NaN
                          Equity(50569 [OUSM])         NaN

[2115083 rows x 1 columns]

In [20]:
# FILTERS
adv10 = AverageDollarVolume(window_length=10)>1e9
top500 = AverageDollarVolume(window_length=10).top(500)

# FACTORS
sma10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10,mask=adv10)
sma20 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=20,mask=adv10)

# CLASSIFIERS
adv_rank = AverageDollarVolume(window_length=10).rank()
adv_quant = AverageDollarVolume(window_length=10).quantiles(10)

In [8]:
import quantopian.pipeline.factors as fact
dir(fact)

['AnnualizedVolatility',
 'Aroon',
 'AverageDollarVolume',
 'BollingerBands',
 'BusinessDaysSincePreviousEvent',
 'BusinessDaysUntilNextEvent',
 'CustomFactor',
 'DailyReturns',
 'EWMA',
 'EWMSTD',
 'ExponentialWeightedMovingAverage',
 'ExponentialWeightedMovingStdDev',
 'Factor',
 'FastStochasticOscillator',
 'IchimokuKinkoHyo',
 'Latest',
 'LinearWeightedMovingAverage',
 'MACDSignal',
 'MarketCap',
 'MaxDrawdown',
 'MovingAverageConvergenceDivergenceSignal',
 'RSI',
 'RateOfChangePercentage',
 'RecarrayField',
 'Returns',
 'RollingLinearRegressionOfReturns',
 'RollingPearsonOfReturns',
 'RollingSpearmanOfReturns',
 'SimpleBeta',
 'SimpleMovingAverage',
 'TrueRange',
 'VWAP',
 'WeightedAverageValue',
 'morningstar']

In [45]:
combined_factor = (sma20-sma10)/sma20

In [4]:
class MyFactor(CustomFactor):
    inputs = [USEquityPricing.close]
    window_length = 10
    mask = adv10
    def compute(self,today,assets,out,close):
        out[:] = close[-1]/close[0]
        
myF = MyFactor()

In [5]:
pipe = Pipeline(columns = {'myfactor':myF},screen=adv10)

In [33]:
from time import time
start = time()
res = run_pipeline(pipe,'2016-01-01','2017-01-01')
print time()-start

2.76629710197


In [34]:
res.head()

myfactor
2016-01-04 00:00:00+00:00 Equity(24 [AAPL])    0.965413
                          Equity(2190 [DIS])   0.937812
                          Equity(3149 [GE])    1.018485
                          Equity(5061 [MSFT])  0.995512
                          Equity(8554 [SPY])   1.001137

In [35]:
dates = res.index.get_level_values(0)
assets = res.index.get_level_values(1)

In [38]:
assets

Index([   Equity(24 [AAPL]),   Equity(2190 [DIS]),    Equity(3149 [GE]),
        Equity(5061 [MSFT]),   Equity(8554 [SPY]), Equity(16841 [AMZN]),
        Equity(19655 [XLE]),  Equity(19920 [QQQ]),  Equity(21519 [IWM]),
        Equity(22972 [EFA]),
       ...
          Equity(24 [AAPL]),    Equity(700 [BAC]),  Equity(5061 [MSFT]),
         Equity(8554 [SPY]), Equity(16841 [AMZN]), Equity(19725 [NVDA]),
        Equity(19920 [QQQ]),  Equity(21519 [IWM]),  Equity(24705 [EEM]),
         Equity(42950 [FB])],
      dtype='object', length=4520)

In [43]:
print res.ix[[(dates[-1])]].index.get_level_values(1)

#for i in range(len(assets)):
#    k = res.ix[[(dates[0],i)]]
#    print k

Index([   Equity(24 [AAPL]),    Equity(700 [BAC]),  Equity(5061 [MSFT]),
         Equity(8554 [SPY]), Equity(16841 [AMZN]), Equity(19725 [NVDA]),
        Equity(19920 [QQQ]),  Equity(21519 [IWM]),  Equity(24705 [EEM]),
         Equity(42950 [FB])],
      dtype='object')


In [21]:
print dates[0]

2016-01-04 00:00:00+00:00


In [4]:
from quantopian.pipeline import Pipeline,CustomFactor
from quantopian.research import run_pipeline
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume
from quantopian.pipeline.data.builtin import USEquityPricing


In [69]:
sma10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10)
adv = AverageDollarVolume(window_length = 10)#.quantiles(100)
fact = adv>99

In [70]:
class MyFactor(CustomFactor):
    def compute(self,today,assets,out,close):
        out[:] = close[-1]/close[0]

myF = MyFactor(window_length=10,inputs=[USEquityPricing.close])

In [71]:
ppl = Pipeline(columns={'myf':myF,'sma':sma10},screen = fact)

In [67]:
res = run_pipeline(ppl,'2016-01-01','2017-01-01')

In [25]:
## TRADABLE UNIVERSE:
from quantopian.pipeline.experimental import QTradableStocksUS
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors.morningstar import MarketCap
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.classifiers.morningstar import Sector

In [7]:
# Note: Zscore is cross-asset not along the factor axis.
my_factor = morningstar.operation_ratios.roe.latest.zscore()

In [11]:
price = USEquityPricing.close.latest
pipe = Pipeline(
        columns={
            'Price': price,
        },
        screen=QTradableStocksUS()
    )

In [24]:
# Pipeline operations
## The resulting summary table displays the mean, std, min-max 
## of daily median in addition to number of assets in this universe:
QTU_pipeline = run_pipeline(pipe, '2018-1-1', '2018-3-1', chunksize=252)
daily_constituent_count = QTU_pipeline.groupby(level=0).sum()
QTU_pipeline.groupby(level=0).median().describe()

,Price
count,41.000000
mean,42.564585
std,1.157521
min,39.741000
25%,42.025000
50%,42.870000
75%,43.570000
max,44.080000


In [9]:
import quantopian.pipeline.data.quandl as quandl
dir(quandl)

['adp_empl_sec',
 'bundesbank_bbk01_wt5511',
 'cboe_rvx',
 'cboe_skew',
 'cboe_vix',
 'cboe_vvix',
 'cboe_vxd',
 'cboe_vxfxi',
 'cboe_vxmt',
 'cboe_vxn',
 'cboe_vxv',
 'cboe_vxxle',
 'chris_cme_cl1',
 'chris_ice_b1',
 'chris_ice_t1',
 'currfx_usdeur',
 'fred_ccsa',
 'fred_dgs10',
 'fred_gdp',
 'fred_gdpc96',
 'fred_gdpdef',
 'fred_gnp',
 'fred_gnpc96',
 'fred_icsa',
 'fred_retailsmnsa',
 'fred_unrate',
 'fred_usdontd156n',
 'oda_usa_ngdp_rpch',
 'rateinf_inflation_usa',
 'ugid_infl_usa',
 'yahoo_index_vix']

### Morningstar Fundamentals

In [22]:
dir(morningstar)
dir(morningstar.balance_sheet)

['accounts_payable',
 'accounts_payable_as_of',
 'accounts_receivable',
 'accounts_receivable_as_of',
 'accrued_interest_receivable',
 'accrued_interest_receivable_as_of',
 'accrued_investment_income',
 'accrued_investment_income_as_of',
 'accrued_liabilities_total',
 'accrued_liabilities_total_as_of',
 'accruedand_deferred_income',
 'accruedand_deferred_income_as_of',
 'accruedand_deferred_income_current',
 'accruedand_deferred_income_current_as_of',
 'accruedand_deferred_income_non_current',
 'accruedand_deferred_income_non_current_as_of',
 'accumulated_depreciation',
 'accumulated_depreciation_as_of',
 'additional_paid_in_capital',
 'additional_paid_in_capital_as_of',
 'advance_from_federal_home_loan_banks',
 'advance_from_federal_home_loan_banks_as_of',
 'advancesfrom_central_banks',
 'advancesfrom_central_banks_as_of',
 'allowance_for_doubtful_accounts_receivable',
 'allowance_for_doubtful_accounts_receivable_as_of',
 'allowance_for_loans_and_lease_losses',
 'allowance_for_loans_a